In [ ]:
### 7.1. Interaktywny edytor subject (wersja minimalna)

from ipywidgets import widgets, VBox, HBox, Output
from IPython.display import display
import pandas as pd
import os

# Wczytanie tabeli kandydatów
df = df_subjects_all.copy()  # df_subjects_all powstaje wcześniej

# Plik z zapisanymi wyborami
save_path = "outputs/csv/subject_human_selected.csv"
os.makedirs("outputs/csv", exist_ok=True)

if os.path.exists(save_path):
    df_sel = pd.read_csv(save_path)
else:
    df_sel = pd.DataFrame(columns=["image_id", "subject_label_pl",
                                   "accepted", "source_module", "score"])

# Lista obrazów
image_ids = sorted(df["image_id"].unique())
dropdown = widgets.Dropdown(options=image_ids, description="Obraz:")

out = Output()

def render_image_subjects(image_id):
    out.clear_output()
    with out:
        sub = df[df["image_id"] == image_id].copy()

        # pobierz poprzednie wybory (jeśli były)
        prev = df_sel[df_sel["image_id"] == image_id]

        checkboxes = []
        for _, row in sub.iterrows():
            label = f"{row['subject_label_pl']} (score={row.get('score', '')}, {row['source_module']})"
            prev_state = False
            if not prev.empty:
                m = prev[prev["subject_label_pl"] == row["subject_label_pl"]]
                if not m.empty:
                    prev_state = bool(m.iloc[0]["accepted"])
            cb = widgets.Checkbox(value=prev_state, description=label)
            checkboxes.append(cb)

        save_btn = widgets.Button(description="Zapisz wybór", button_style="success")

        def save_clicked(_):
            nonlocal df_sel
            records = []
            for cb, (_, row) in zip(checkboxes, sub.iterrows()):
                records.append({
                    "image_id": row["image_id"],
                    "subject_label_pl": row["subject_label_pl"],
                    "accepted": cb.value,
                    "source_module": row["source_module"],
                    "score": row.get("score", None),
                })
            # usuń stare rekordy tego obrazu i dodaj nowe
            df_sel = df_sel[df_sel["image_id"] != image_id]
            df_sel = pd.concat([df_sel, pd.DataFrame(records)], ignore_index=True)
            df_sel.to_csv(save_path, index=False)
            print("Zapisano.")

        save_btn.on_click(save_clicked)

        display(VBox(checkboxes + [save_btn]))

def on_select(change):
    render_image_subjects(change["new"])

dropdown.observe(on_select, names="value")
display(dropdown, out)

# uruchom dla pierwszego obrazu
render_image_subjects(image_ids[0])

### Końcowy eksport

In [ ]:
### 7.2. Eksport do Dublin Core

df_accept = pd.read_csv("outputs/csv/subject_human_selected.csv")
df_accept = df_accept[df_accept["accepted"] == True]

df_dc = (
    df_accept.groupby("image_id")["subject_label_pl"]
    .apply(lambda vals: "; ".join(sorted(set(vals))))
    .reset_index()
    .rename(columns={"image_id": "identifier", "subject_label_pl": "subject"})
)

df_dc.to_csv("outputs/csv/dublin_core_export.csv", index=False)
"outputs/csv/dublin_core_export.csv"